In [0]:
%pip install yahooquery

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import pickle
import requests
import numpy as np
from yahooquery import Ticker
import datetime
from datetime import date
import urllib.request
import sys
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import re

In [0]:
%run "/Shared/WeatherWatts/config/kafka keys"

In [0]:
%run "/Shared/WeatherWatts/config/api keys"

In [0]:
delta1 = datetime.timedelta(days=2)
delta2  =datetime.timedelta(days=4)
delta3 = datetime.timedelta(days=-3)
delta4 = datetime.timedelta(days=1)
delta5 = datetime.timedelta(days=3)
today = date.today()
one_day_ahead = today + delta4
two_day_ahead = today + delta1
four_days_ahead = today + delta2
three_days_behind = today +delta3
three_days_ahead = today + delta5

In [0]:
holidays = spark.read.csv('/mnt/weatherwatts/holiday.csv', header=True)

In [0]:
holidays = holidays.toPandas()

In [0]:
ticker = Ticker('XLE')

In [0]:
XLE = ticker.history(period='3mo', interval='1d', adj_timezone=False)['adjclose'].unstack().T

In [0]:
future = pd.date_range(str(one_day_ahead), str(four_days_ahead), freq='1d', tz='UTC')

In [0]:
future_df = pd.DataFrame(index=future)

In [0]:
XLE = pd.concat([XLE, future_df])

In [0]:
from datetime import date
XLE.index = pd.to_datetime(XLE.index, utc=True)
XLE['year'] = XLE.index.year
XLE['dayofyear'] =XLE.index.dayofyear

In [0]:
XLE.rename(columns={'XLE':'Adj Close'}, inplace=True)

In [0]:
XLE['Adj_Close_3D_lag'] = XLE['Adj Close'].shift(freq='3D')
XLE['Adj_Close_30D_lag'] = XLE['Adj Close'].shift(freq='28D')
XLE['Adj_Close_60D_lag'] = XLE['Adj Close'].shift(freq='56D')
XLE['Adj_Close_30D_lag'] = XLE['Adj_Close_30D_lag'].ffill()
XLE['Adj_Close_60D_lag'] = XLE['Adj_Close_60D_lag'].ffill()
XLE['Adj_Close_3D_lag'] = XLE['Adj_Close_3D_lag'].ffill()


In [0]:
locations2 = ['Los%20Angeles', 'Tampa', 'Austin','new%20york%20city']

In [0]:
weather_df = pd.DataFrame()
try:
    for loc in locations2:
        ResultBytes = urllib.request.urlopen(f"https://weather.visualcrossing.com/VisualCrossingWebServices/"
                                             f"rest/services/timeline/{loc}/{three_days_behind}/{four_days_ahead}?"
                                             f"unitGroup=us&include=fcst%2Chours%2Cobs&"
                                             f"key={w_key}&contentType=json")

      # Parse the results as JSON
        jsonData = json.load(ResultBytes)
        for i in range(len(jsonData['days'])):
            day = pd.DataFrame(jsonData['days'][i]['hours'])
            day['name'] = loc.replace('%20',' ')
            weather_df = pd.concat([weather_df,day])

except urllib.error.HTTPError  as e:
  ErrorInfo= e.read().decode()
  print('Error code: ', e.code, ErrorInfo)
  sys.exit()
except  urllib.error.URLError as e:
  ErrorInfo= e.read().decode()
  print('Error code: ', e.code,ErrorInfo)
  sys.exit()

In [0]:
locations =[ "CISO", "ERCO","NYIS","FLA"]

In [0]:
def json_to_dateframe(response):
    return pd.DataFrame(response.json()['response']['data'][1:], columns= response.json()['response']['data'][0])

In [0]:
from time import sleep
energy = pd.DataFrame()
for loc in locations:
    try:
        url = f'https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&facets[respondent][]={loc}&facets[type][]=D&facets[type][]=NG&start={str(three_days_behind)}T00&end={str(today)}T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={key}'
        response = requests.get(url)
        df = json_to_dateframe(response)
        energy = pd.concat([energy, df])
    except:
        print('error')

In [0]:
demand = energy.query("type == 'D' ")

In [0]:
demand['respondent-name'] = demand['respondent-name'].replace('Florida', 'Florida Power and Light').copy()
demand['period'] = pd.to_datetime(demand.period)
demand['period'] = pd.to_datetime(demand.period)

In [0]:
respondents =['Electric Reliability Council of Texas, Inc.' ,
              'New York Independent System Operator',
              'Florida Power and Light' ,
               'California Independent System Operator',]

In [0]:
demand['name'] = demand['respondent-name']

In [0]:
demand['name'] = demand['name'].replace(respondents[3], 'Los Angeles')
demand['name'] = demand['name'].replace(respondents[2], 'Tampa')
demand['name'] = demand['name'].replace(respondents[0], 'Austin')
demand['name'] = demand['name'].replace(respondents[1], 'new york city')

In [0]:
weather_df['period'] = pd.to_datetime(weather_df.datetimeEpoch, unit='s')

In [0]:
demand_weather = pd.merge(demand,weather_df, how='right', on=['name','period'],suffixes=('','2'))
demand_weather = demand_weather.set_index(demand_weather.period)
demand_weather= demand_weather.drop(columns=['type','type-name','datetime','datetimeEpoch','stations','solarenergy','source','severerisk','preciptype','windgust','value-units','respondent','icon','conditions'])

In [0]:
demand_weather['year'] = demand_weather.index.year
demand_weather['month'] = demand_weather.index.month
demand_weather['days'] = demand_weather.index.dayofweek
demand_weather['hour'] = demand_weather.index.hour
demand_weather['quarter'] = demand_weather.index.quarter
demand_weather['dayofyear'] = demand_weather.index.dayofyear
demand_weather["Is Weekend"] = demand_weather.index.day_name().isin(['Saturday', 'Sunday'])

In [0]:
demand_weather = pd.merge(demand_weather,XLE.iloc[:,1:],how='left', on=['year','dayofyear'])

In [0]:
holidays['year'] = holidays['year'].astype('int')
holidays['dayofyear'] = holidays['dayofyear'].astype('int')

In [0]:
holidays =holidays.drop('Date',axis=1)
demand_weather = pd.merge(demand_weather,holidays, how='outer', on=['year','dayofyear'])

In [0]:
demand_weather['solarradiation'] = demand_weather['solarradiation'].bfill()
demand_weather['uvindex'] = demand_weather['uvindex'].bfill()
demand_weather['snow'] = demand_weather['snow'].bfill()

In [0]:
demand_weather.loc[demand_weather['name'] == 'Los Angeles', 'respondent-name'] = 'California Independent System Operator'
demand_weather.loc[demand_weather['name'] == 'new york city', 'respondent-name'] = 'New York Independent System Operator'
demand_weather.loc[demand_weather['name'] == 'Tampa', 'respondent-name'] = 'Florida Power and Light'
demand_weather.loc[demand_weather['name'] == 'Austin', 'respondent-name'] = 'Electric Reliability Council of Texas, Inc.'

In [0]:
demand_weather = demand_weather.set_index('period')
demand_weather.index = pd.to_datetime(demand_weather.index)
demand_weather.rename(columns={'respondent-name':'respondent','pressure': 'sealevelevelpressure'}, inplace=True)

In [0]:
demand_weather = pd.get_dummies(demand_weather, columns=['Holiday','Is Weekend'], drop_first=True, dtype=float)
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in demand_weather.columns}
new_n_list = list(new_names.values())
# [LightGBM] Feature appears more than one time.
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
demand_weather = demand_weather.rename(columns=new_names)

In [0]:
NY = demand_weather.query("respondent== 'New York Independent System Operator'")
FL = demand_weather.query("respondent == 'Florida Power and Light'")
CA = demand_weather.query("respondent ==  'California Independent System Operator'")
TX = demand_weather.query("respondent ==  'Electric Reliability Council of Texas, Inc.'")

In [0]:
TX = TX[~TX.index.duplicated(keep='first')]
NY = NY[~NY.index.duplicated(keep='first')]
CA = CA[~CA.index.duplicated(keep='first')]
FL = FL[~FL.index.duplicated(keep='first')]

In [0]:
NY['demand_3d_lag'] = NY['value'].shift(96, freq='h')
FL['demand_3d_lag'] = FL['value'].shift(96, freq='h')
CA['demand_3d_lag'] = CA['value'].shift(96, freq='h')
TX['demand_3d_lag'] = TX['value'].shift(96, freq='h')

In [0]:
NY['Temp_24_hours_ahead'] = NY['temp'].shift(-24, freq='h')
FL['Temp_24_hours_ahead'] = FL['temp'].shift(-24 ,freq='h')
CA['Temp_24_hours_ahead'] = CA['temp'].shift(-24, freq='h')
TX['Temp_24_hours_ahead'] = TX['temp'].shift(-24, freq='h')

In [0]:
# TX  = TX.iloc[(TX.index >= str(today)) & (TX.index <= str(four_days_ahead))]
# NY  = NY.iloc[(NY.index >= str(today)) & (NY.index <= str(four_days_ahead))]
# CA  = CA.iloc[(CA.index >= str(today)) & (CA.index <= str(four_days_ahead))]
# FL  = FL.iloc[(FL.index >= str(today)) & (FL.index <= str(four_days_ahead))]

In [0]:
TX  = TX.iloc[(TX.index > str(three_days_ahead))][:24]
NY  = NY.iloc[(NY.index > str(three_days_ahead))][:24]
CA  = CA.iloc[(CA.index > str(three_days_ahead))][:24]
FL  = FL.iloc[(FL.index > str(three_days_ahead))][:24]

In [0]:
Features = TX.columns[3:]

In [0]:
TX_X = TX[Features]
NY_X = NY[Features]
CA_X = CA[Features]
FL_X = FL[Features]

In [0]:
TX_X = TX_X.ffill()
NY_X = NY_X.ffill()
CA_X = CA_X.ffill()
FL_X = FL_X.ffill()

In [0]:
TX_X = TX_X.dropna(subset=['demand_3d_lag'])
NY_X = NY_X.dropna(subset=['demand_3d_lag'])
CA_X = CA_X.dropna(subset=['demand_3d_lag'])
FL_X = FL_X.dropna(subset=['demand_3d_lag'])

In [0]:
TX_X['State'] = 'TX'
NY_X['State'] = 'NY'
CA_X['State'] = 'CA'
FL_X['State'] = 'FL'

In [0]:
final_df = pd.concat([TX_X,NY_X,FL_X,CA_X])

In [0]:
final_df['Date'] = final_df.index

In [0]:
final_df = spark.createDataFrame(final_df)

In [0]:
display(final_df)

temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,windspeed,winddir,sealevelevelpressure,visibility,cloudcover,solarradiation,uvindex,year,month,days,hour,quarter,dayofyear,Adj_Close_3D_lag,Adj_Close_30D_lag,Adj_Close_60D_lag,Holiday_ChristmasDay,Holiday_ChristmasDaysubstitute,Holiday_ChristmasEve,Holiday_CincodeMayo,Holiday_ColumbusDay,Holiday_DayAfterChristmasDay,Holiday_EasterMonday,Holiday_EasterSunday,Holiday_ElectionDay,Holiday_ElectionDayGeneralElection,Holiday_FathersDay,Holiday_FlagDay,Holiday_Halloween,Holiday_InaugurationDay,Holiday_IndependenceDay,Holiday_IndependenceDaysubstitute,Holiday_Juneteenth,Holiday_Juneteenthsubstitute,Holiday_LaborDay,Holiday_MartinLutherKingJrDay,Holiday_MemorialDay,Holiday_MothersDay,Holiday_NewYearsDay,Holiday_NewYearsDaysubstitute,Holiday_NewYearsEve,Holiday_PresidentsDay,Holiday_StPatricksDay,Holiday_SuperTuesday,Holiday_TaxDay,Holiday_ThanksgivingDay,Holiday_ValentinesDay,Holiday_VeteransDay,Holiday_VeteransDaysubstitute,IsWeekend_True,demand_3d_lag,Temp_24_hours_ahead,State,Date
75.6,75.6,60.99,61.2,0.0,10.0,0.0,0.0,4.5,60.0,1016.0,10.1,25.3,103.0,1.0,2023,5.0,0.0,1.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58282.0,80.0,TX,2023-05-22T01:00:00.000+0000
73.5,73.5,67.25,62.0,0.0,10.0,0.0,0.0,3.6,70.0,1016.0,10.1,20.7,64.0,0.0,2023,5.0,0.0,2.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56281.0,77.4,TX,2023-05-22T02:00:00.000+0000
71.3,71.3,73.73,62.5,0.0,10.0,0.0,0.0,2.7,80.0,1016.0,10.1,16.0,26.0,0.0,2023,5.0,0.0,3.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54137.0,74.7,TX,2023-05-22T03:00:00.000+0000
69.7,69.7,77.42,62.3,0.0,10.0,0.0,0.0,2.7,60.0,1017.0,10.1,15.3,22.0,0.0,2023,5.0,0.0,4.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50341.0,73.3,TX,2023-05-22T04:00:00.000+0000
68.3,68.3,80.3,62.0,0.0,10.0,0.0,0.0,2.7,40.0,1017.0,10.1,14.7,17.0,0.0,2023,5.0,0.0,5.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46216.0,71.7,TX,2023-05-22T05:00:00.000+0000
66.6,66.6,84.38,61.8,0.0,10.0,0.0,0.0,2.7,20.0,1017.0,10.1,14.0,13.0,0.0,2023,5.0,0.0,6.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42909.0,70.2,TX,2023-05-22T06:00:00.000+0000
65.9,65.9,85.42,61.4,0.0,4.0,0.0,0.0,2.7,20.0,1016.0,10.1,14.3,9.0,0.0,2023,5.0,0.0,7.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40698.0,69.3,TX,2023-05-22T07:00:00.000+0000
65.0,65.0,87.01,61.1,0.0,4.0,0.0,0.0,2.7,20.0,1016.0,10.1,14.7,4.0,0.0,2023,5.0,0.0,8.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39198.0,68.6,TX,2023-05-22T08:00:00.000+0000
64.3,64.3,88.1,60.7,0.0,4.0,0.0,0.0,2.7,20.0,1016.0,10.1,15.0,0.0,0.0,2023,5.0,0.0,9.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3825

In [0]:
from pyspark.sql.functions import *
import json
json_final = final_df.toJSON().map(lambda str_json: json.loads(str_json)).collect()

In [0]:
type(json_final)

Out[53]: list

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
# from confluent_kafka.admin import AdminClient
# #creating an admin_client object
# admin_client = AdminClient({
#     'bootstrap.servers': confluentBootstrapServers,
#     'sasl.mechanism': 'PLAIN',
#     'security.protocol': 'SASL_SSL',
#     'sasl.username': confluentApiKey,
#     'sasl.password': confluentSecret,
#     'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
#     'auto.offset.reset': 'earliest',
#     'error_cb': error_cb,
# })

In [0]:
# from confluent_kafka.admin import NewTopic

# topic_list = []

# topic_list.append(NewTopic(confluentTopicName, 1, 3))
# # admin_client.create_topics(topic_list)
# futures = admin_client.create_topics(topic_list)

# try:
#     record_metadata = []
#     for k, future in futures.items():
#         # f = i.get(timeout=10)
#         print(f"type(k): {type(k)}")
#         print(f"type(v): {type(future)}")
#         print(future.result())

# except KafkaError:
#     # Decide what to do if produce request failed...
#     print(traceback.format_exc())
#     result = 'Fail'
# finally:
#     print("finally")

In [0]:
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
len(json_final)

Out[58]: 96

In [0]:
for i in range(len(json_final)): 
    # print(json_final[i])
    p.produce(confluentTopicName,json.dumps(json_final[i]))
    p.flush()

In [0]:
display(final_df)

temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,windspeed,winddir,sealevelevelpressure,visibility,cloudcover,solarradiation,uvindex,year,month,days,hour,quarter,dayofyear,Adj_Close_3D_lag,Adj_Close_30D_lag,Adj_Close_60D_lag,Holiday_ChristmasDay,Holiday_ChristmasDaysubstitute,Holiday_ChristmasEve,Holiday_CincodeMayo,Holiday_ColumbusDay,Holiday_DayAfterChristmasDay,Holiday_EasterMonday,Holiday_EasterSunday,Holiday_ElectionDay,Holiday_ElectionDayGeneralElection,Holiday_FathersDay,Holiday_FlagDay,Holiday_Halloween,Holiday_InaugurationDay,Holiday_IndependenceDay,Holiday_IndependenceDaysubstitute,Holiday_Juneteenth,Holiday_Juneteenthsubstitute,Holiday_LaborDay,Holiday_MartinLutherKingJrDay,Holiday_MemorialDay,Holiday_MothersDay,Holiday_NewYearsDay,Holiday_NewYearsDaysubstitute,Holiday_NewYearsEve,Holiday_PresidentsDay,Holiday_StPatricksDay,Holiday_SuperTuesday,Holiday_TaxDay,Holiday_ThanksgivingDay,Holiday_ValentinesDay,Holiday_VeteransDay,Holiday_VeteransDaysubstitute,IsWeekend_True,demand_3d_lag,Temp_24_hours_ahead,State,Date
75.6,75.6,60.99,61.2,0.0,10.0,0.0,0.0,4.5,60.0,1016.0,10.1,25.3,103.0,1.0,2023,5.0,0.0,1.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58282.0,80.0,TX,2023-05-22T01:00:00.000+0000
73.5,73.5,67.25,62.0,0.0,10.0,0.0,0.0,3.6,70.0,1016.0,10.1,20.7,64.0,0.0,2023,5.0,0.0,2.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56281.0,77.4,TX,2023-05-22T02:00:00.000+0000
71.3,71.3,73.73,62.5,0.0,10.0,0.0,0.0,2.7,80.0,1016.0,10.1,16.0,26.0,0.0,2023,5.0,0.0,3.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54137.0,74.7,TX,2023-05-22T03:00:00.000+0000
69.7,69.7,77.42,62.3,0.0,10.0,0.0,0.0,2.7,60.0,1017.0,10.1,15.3,22.0,0.0,2023,5.0,0.0,4.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50341.0,73.3,TX,2023-05-22T04:00:00.000+0000
68.3,68.3,80.3,62.0,0.0,10.0,0.0,0.0,2.7,40.0,1017.0,10.1,14.7,17.0,0.0,2023,5.0,0.0,5.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46216.0,71.7,TX,2023-05-22T05:00:00.000+0000
66.6,66.6,84.38,61.8,0.0,10.0,0.0,0.0,2.7,20.0,1017.0,10.1,14.0,13.0,0.0,2023,5.0,0.0,6.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42909.0,70.2,TX,2023-05-22T06:00:00.000+0000
65.9,65.9,85.42,61.4,0.0,4.0,0.0,0.0,2.7,20.0,1016.0,10.1,14.3,9.0,0.0,2023,5.0,0.0,7.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40698.0,69.3,TX,2023-05-22T07:00:00.000+0000
65.0,65.0,87.01,61.1,0.0,4.0,0.0,0.0,2.7,20.0,1016.0,10.1,14.7,4.0,0.0,2023,5.0,0.0,8.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39198.0,68.6,TX,2023-05-22T08:00:00.000+0000
64.3,64.3,88.1,60.7,0.0,4.0,0.0,0.0,2.7,20.0,1016.0,10.1,15.0,0.0,0.0,2023,5.0,0.0,9.0,2.0,142,79.02999877929688,86.27999877929688,79.55000305175781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3825

In [0]:
CA

,respondent,value,name,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,windspeed,winddir,sealevelevelpressure,visibility,cloudcover,solarradiation,uvindex,year,month,days,hour,quarter,dayofyear,Adj_Close_3D_lag,Adj_Close_30D_lag,Adj_Close_60D_lag,Holiday_ChristmasDay,Holiday_ChristmasDaysubstitute,Holiday_ChristmasEve,Holiday_CincodeMayo,Holiday_ColumbusDay,Holiday_DayAfterChristmasDay,Holiday_EasterMonday,Holiday_EasterSunday,Holiday_ElectionDay,Holiday_ElectionDayGeneralElection,Holiday_FathersDay,Holiday_FlagDay,Holiday_Halloween,Holiday_InaugurationDay,Holiday_IndependenceDay,Holiday_IndependenceDaysubstitute,Holiday_Juneteenth,Holiday_Juneteenthsubstitute,Holiday_LaborDay,Holiday_MartinLutherKingJrDay,Holiday_MemorialDay,Holiday_MothersDay,Holiday_NewYearsDay,Holiday_NewYearsDaysubstitute,Holiday_NewYearsEve,Holiday_PresidentsDay,Holiday_StPatricksDay,Holiday_SuperTuesday,Holiday_TaxDay,Holiday_ThanksgivingDay,Holiday_ValentinesDay,Holiday_VeteransDay,Holiday_VeteransDaysubstitute,IsWeekend_True,demand_3d_lag,Temp_24_hours_ahead
period,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-22 01:00:00,California Independent System Operator,NaN,Los Angeles,69.7,69.7,65.54,57.6,0.0,3.0,0.0,0.0,11.4,226.7,1013.0,10.1,18.0,656.0,7.0,2023,5.0,0.0,1.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27231.0,66.6
2023-05-22 02:00:00,California Independent System Operator,NaN,Los Angeles,67.2,67.2,71.46,57.6,0.0,3.0,0.0,0.0,10.1,223.3,1013.0,10.1,22.0,437.0,4.0,2023,5.0,0.0,2.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28338.0,64.1
2023-05-22 03:00:00,California Independent System Operator,NaN,Los Angeles,64.8,64.8,77.00,57.5,0.0,3.0,0.0,0.0,8.9,220.0,1014.0,10.1,26.0,219.0,0.0,2023,5.0,0.0,3.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29371.0,61.8
2023-05-22 04:00:00,California Independent System Operator,NaN,Los Angeles,63.9,63.9,79.46,57.5,0.0,3.0,0.0,0.0,7.8,206.7,1014.0,10.1,36.3,182.0,0.0,2023,5.0,0.0,4.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29987.0,61.2
2023-05-22 05:00:00,California Independent System Operator,NaN,Los Angeles,63.2,63.2,81.49,57.5,0.0,3.0,0.0,0.0,6.5,193.3,1015.0,10.1,46.7,146.0,0.0,2023,5.0,0.0,5.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30092.0,60.7
2023-05-22 06:00:00,California Independent System Operator,NaN,Los Angeles,62.3,62.3,84.11,57.5,0.0,3.0,0.0,0.0,5.4,180.0,1015.0,10.1,57.0,109.0,0.0,2023,5.0,0.0,6.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29234.0,60.2
2023-05-22 07:00:00,California Independent System Operator,NaN,Los Angeles,61.8,61.8,86.29,57.6,0.0,2.0,0.0,0.0,4.7,176.7,1015.0,10.1,57.0,73.0,0.0,2023,5.0,0.0,7.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27463.0,59.8
2023-05-22 08:00:00,California Independent System Operator,NaN,Los Angeles,61.2,61.2,87.95,57.6,0.0,2.0,0.0,0.0,4.3,173.3,1014.0,10.1,57.0,36.0,0.0,2023,5.0,0.0,8.0,2.0,142,79.029999,86.279999,79.550003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25941.0,59.6
2023-05-22 09:00:00,California Independent 